#### Finetune GPT2 with AVA-Captions dataset

In [1]:
import os
import shutil
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import pipeline

Used code from: https://rowlando13.medium.com/everything-gpt-2-5-fine-tuning-885aec508c4

In [2]:
model_path = "../CLIP-GPT2/models/gpt2-large"
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

In [3]:
def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=train_path,
        block_size=128,
    )
    test_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=test_path,
        block_size=128,
    )
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

train_path = "../CLIP-GPT2/data/large/train.txt"
test_path = "../CLIP-GPT2/data/large/test.txt"
train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

c:\Users\Keenan Samway\miniconda3\envs\pytorch\lib\site-packages\transformers\data\datasets\language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [8]:
data_collator

DataCollatorForLanguageModeling(tokenizer=PreTrainedTokenizer(name_or_path='../CLIP-GPT2/models/gpt2-large', vocab_size=50257, model_max_len=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True)}), mlm=False, mlm_probability=0.15, pad_to_multiple_of=None, tf_experimental_compile=False, return_tensors='pt')

In [7]:
test_dataset[2]

tensor([39113,    13,  1201,   428,   318,  6190, 12857,  1312,   892,  1312,
          561,   466,  1223,  3863, 21792,  9482,   284,   644,  3568,   284,
          307,  6268, 23671,   319,   262,  1353,  8470,    13,  3073,   588,
         7862,  8470,   284,   502,    13,  3621,  3124, 12019,    13,  7862,
         8470,     0, 20105,    11,  1312,   561,  1842,   284,  3285,   511,
         3496,     0,  1312,  1842, 19974,  8470,   612,   826,   510,   612,
          351, 28494,  9154,  1276,   307,   617,  1611,   286,  5417,  7185,
           13,   428,   329,   617,  1738, 17603,   502,   286,   257, 24276,
        26842,    13, 44929,   286,   257,  6228, 11084, 26842,    30,  4168,
          286,  1223,    13,  3297,   286, 23387,     0,   262, 44929,   286,
          257, 26842,  5417,  2099,   262,  3793,   286,   257,  2636,  5156,
        26842,    30,  3863,   257, 22647,   393,   257, 26842,   845,  1862,
           13,  2508, 15756, 27179,  9124,    30, 29181,    11])

In [4]:
training_args = TrainingArguments(
    output_dir="../CLIP-GPT2/models/gpt2-large-AVA",
    overwrite_output_dir=False,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=1,
    eval_steps=5000,
    save_steps=10000,
    warmup_steps=500,
    fp16=True,
    fp16_opt_level="O1",
    prediction_loss_only=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

Using cuda_amp half precision backend


In [5]:
trainer.train(resume_from_checkpoint=True) 

Loading model from ../CLIP-GPT2/models/gpt2-large-AVA\checkpoint-20000.
c:\Users\Keenan Samway\miniconda3\envs\pytorch\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 219333
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 82251
  Number of trainable parameters = 774030080
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 0
  Continuing training from global step 20000
  Will skip the first 0 epochs then the first 20000 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` f

  0%|          | 0/20000 [00:00<?, ?it/s]

  0%|          | 0/82251 [00:00<?, ?it/s]

{'loss': 3.2334, 'learning_rate': 3.7772015021222984e-05, 'epoch': 0.75}
{'loss': 3.2254, 'learning_rate': 3.746620836442368e-05, 'epoch': 0.77}
{'loss': 3.2262, 'learning_rate': 3.7160401707624375e-05, 'epoch': 0.78}
{'loss': 3.228, 'learning_rate': 3.685459505082507e-05, 'epoch': 0.8}
{'loss': 3.2226, 'learning_rate': 3.654878839402576e-05, 'epoch': 0.82}
{'loss': 3.2257, 'learning_rate': 3.624359335054006e-05, 'epoch': 0.84}
{'loss': 3.2301, 'learning_rate': 3.593778669374075e-05, 'epoch': 0.86}
{'loss': 3.2246, 'learning_rate': 3.563198003694145e-05, 'epoch': 0.88}
{'loss': 3.2214, 'learning_rate': 3.532617338014214e-05, 'epoch': 0.89}
{'loss': 3.219, 'learning_rate': 3.5020366723342835e-05, 'epoch': 0.91}
{'loss': 3.2168, 'learning_rate': 3.471456006654353e-05, 'epoch': 0.93}
{'loss': 3.2179, 'learning_rate': 3.4408753409744226e-05, 'epoch': 0.95}
{'loss': 3.2115, 'learning_rate': 3.410294675294492e-05, 'epoch': 0.97}
{'loss': 3.2056, 'learning_rate': 3.3797751709459214e-05, 'epoc

Saving model checkpoint to ../CLIP-GPT2/models/gpt2-large-AVA\checkpoint-30000
Configuration saved in ../CLIP-GPT2/models/gpt2-large-AVA\checkpoint-30000\config.json


...

Model weights saved in ../CLIP-GPT2/models/gpt2-large-AVA\checkpoint-80000\pytorch_model.bin


{'loss': 2.8581, 'learning_rate': 1.0886716982055265e-06, 'epoch': 2.94}
{'loss': 2.8443, 'learning_rate': 7.828650414062213e-07, 'epoch': 2.95}
{'loss': 2.8479, 'learning_rate': 4.770583846069161e-07, 'epoch': 2.97}
{'loss': 2.8435, 'learning_rate': 1.7125172780761094e-07, 'epoch': 2.99}




Training completed. Do not forget to share your model on huggingface.co/models =)




{'train_runtime': 19054.3297, 'train_samples_per_second': 34.533, 'train_steps_per_second': 4.317, 'train_loss': 2.258287075406473, 'epoch': 3.0}


TrainOutput(global_step=82251, training_loss=2.258287075406473, metrics={'train_runtime': 19054.3297, 'train_samples_per_second': 34.533, 'train_steps_per_second': 4.317, 'train_loss': 2.258287075406473, 'epoch': 3.0})

# Reset Runtime

In [1]:
import os
import shutil
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import pipeline

In [2]:
model_path = "../CLIP-GPT2/models/gpt2-large"
tokenizer = GPT2Tokenizer.from_pretrained(model_path)

m_loc = "../CLIP-GPT2/models/gpt2_large-AVA/checkpoint-80000"
model = GPT2LMHeadModel.from_pretrained(m_loc)

In [4]:
prefix = "I like this image, but "
tokens = tokenizer.encode(prefix)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
output = pipe(prefix, max_new_tokens=50, num_return_sequences=5, pad_token_id=50256, num_beams=5)

output

[{'generated_text': "I like this image, but ive seen a lot of them. i think this is the best of the lot. i really like the composition and the colors. the only thing i don't like is the border. i think it takes away from the simplicity of the shot. i"},
 {'generated_text': 'I like this image, but ive seen so many of them now that its starting to get a little boring. this is a nice image. i really like the angle from which you shot this image. i also like the fact that you shot this image in black and white. nice'},
 {'generated_text': 'I like this image, but ive never been able to pull it off as well as you have here. i love the way the light is hitting the top of the glass, and the way it reflects on the table below. great job. i really like this one. the only'},
 {'generated_text': 'I like this image, but ive never been able to pull it off. great job. this is a great idea, but i think it would have been better if you could have gotten rid of the shadow in the bottom of the picture. g